In [1]:
import time
from datetime import datetime
from pprint import pprint as pp

import fireducks.pandas as pd
import pygwalker as pyg
from api_binance import get_klines
from api_bitpreco import (
    Balance,
    OpenOrders,
    Orderbook,
    Ticker,
    Trades,
    fetch_bitpreco_history,
    get_coinpair,
)
from estrategias import BTCBRL_BITY, BTCBRL_FILE
from rich import print

# Tabela da ordem de compra e venda executadas
from segredos import CAMINHO


In [ ]:
print(datetime.now().astimezone().timetz().tzinfo)
# dois dias atrás em timestamp
print(datetime.now().timestamp() - 2 * 24 * 60 * 60)

In [ ]:
response = get_klines()

display(response.columns)

# Minhas Ordens executadas

In [ ]:
coinpair = get_coinpair()
coinpair = 'all-brl'
pp(Ticker(coinpair).json().keys())

In [ ]:
balance = Balance().json()
# pp(balance)
balance_df = pd.DataFrame([balance])

balance_df

In [6]:
open_orders = OpenOrders().json()

if open_orders != []:
    open_orders_df = pd.DataFrame(open_orders)
    pyg.walk(open_orders_df)

In [ ]:
def calculate_fee(
    order_type: str, origin: str, amount: float, price: float
) -> float:
    """
    Calcula a taxa da ordem baseado no tipo, origem e valor.

    Args:
        order_type: Tipo da ordem ('BUY' ou 'SELL')
        origin: Origem da ordem ('API', 'APP' ou 'WEB')
        amount: Quantidade de BTC
        price: Preço unitário em BRL

    Returns:
        float: Valor da taxa em BTC
    """
    valor_total = amount * price

    # Taxa para ordens de venda (SELL)
    if order_type == 'SELL':
        if origin in {'API', 'WEB'}:
            return amount * 0.002  # 0.2%
        elif origin == 'APP':
            return amount * 0.002  # 0.2%

    # Taxa para ordens de compra (BUY)
    elif order_type == 'BUY':
        if origin == 'API':
            return 2e-8 if valor_total < 50 else 5e-7  # noqa: PLR2004
        elif origin == 'APP':
            return 0.0  # Algumas ordens APP têm taxa zero
        elif origin == 'WEB':
            return 0.0  # Algumas ordens WEB têm taxa zero

    return 0.0

In [ ]:
ORDERS_FILE = CAMINHO + '/executed_orders.csv'

executed_orders_df = pd.read_csv(ORDERS_FILE)
# pegando todas as ordens de compra até a ultima ordem de venda
# Filtra ordens executadas até a última ordem de venda
last_sell_index = executed_orders_df[
    executed_orders_df['type'] == 'SELL'
].index[0]
filtered_orders = executed_orders_df.loc[:last_sell_index]

# Filtra apenas ordens de compra
buy_orders = filtered_orders[filtered_orders['type'] == 'BUY']
print(executed_orders_df)
print(buy_orders)

# criando uma lista de amount que ainda precisam ser
# vendidos com seus respectivos preços e taxas
amounts_to_sell = []


id   market  type  status    amount          price  exec_amount  \
0   1520875822  BTC-BRL   BUY  FILLED  0.000007  601200.000000     0.000007   
1   1520875579  BTC-BRL   BUY  FILLED  0.000009  601200.000000     0.000009   
2   1504227287  BTC-BRL   BUY  FILLED  0.000010  648107.029703     0.000010   
3   1504205222  BTC-BRL   BUY  FILLED  0.000013  650346.979332     0.000013   
4   1503401599  BTC-BRL   BUY  FILLED  0.000016  651822.016592     0.000016   
5   1502873638  BTC-BRL   BUY  FILLED  0.000019  660740.000000     0.000019   
6   1502871460  BTC-BRL   BUY  FILLED  0.000024  662639.004149     0.000024   
7   1502870034  BTC-BRL   BUY  FILLED  0.000030  663701.995344     0.000030   
8   1501991376  BTC-BRL  SELL  FILLED  0.000154  649247.000779     0.000154   
9   1501767741  BTC-BRL   BUY  FILLED  0.000154  652086.036095     0.000154   
10  1498589676  BTC-BRL  SELL  FILLED  0.000165  608500.000000     0.000165   
11  1498588089  BTC-BRL   BUY  FILLED  0.000166  606898.998371     0.000166   
12  1432828055  BTC-BRL  SELL  FILLED  0.000070  367698.993819     0.000070   
13  1417883177  BTC-BRL   BUY  FILLED  0.000070  358728.794835     0.000070   
14  1413970020  BTC-BRL  SELL  FILLED  0.000278  360000.000000     0.000278   
15  1412867104  BTC-BRL   BUY  FILLED  0.000278  353500.000000     0.000278   
16  1412311243  BTC-BRL  SELL  FILLED  0.000280  352000.000000     0.000280   
17  1411474267  BTC-BRL   BUY  FILLED  0.000281  348983.000427     0.000281   
18  1411095452  BTC-BRL  SELL  FILLED  0.000282  348500.000000     0.000282   
19  1410548290  BTC-BRL   BUY  FILLED  0.000283  345937.991868     0.000283   
20  1409138168  BTC-BRL  SELL  FILLED  0.000012  346000.000000     0.000012   
21  1409136629  BTC-BRL   BUY  FILLED  0.000012  343101.974249     0.000012   
22  1408949034  BTC-BRL  SELL  FILLED  0.000071  341406.000000     0.000071   
23  1408948406  BTC-BRL   BUY  FILLED  0.000071  340809.994377     0.000071   
24  1408942491  BTC-BRL  SELL  FILLED  0.000071  339500.000000     0.000071   
25  1408933303  BTC-BRL   BUY  FILLED  0.000071  339316.994658     0.000071   
26  1408074126  BTC-BRL  SELL  FILLED  0.000300  327352.852903     0.000300   
27  1407326572  BTC-BRL   BUY  FILLED  0.000071  330548.003936     0.000071   
28  1407146150  BTC-BRL   BUY  FILLED  0.000229  336975.000000     0.000229   
29  1407143157  BTC-BRL  SELL  FILLED  0.000229  337077.991453     0.000229   
30  1407130874  BTC-BRL   BUY  FILLED  0.000077  336367.006137     0.000077   
31  1407130046  BTC-BRL   BUY  FILLED  0.000077  336947.995822     0.000077   
32  1407127071  BTC-BRL   BUY  FILLED  0.000077  337077.999739     0.000077   
33  1407126428  BTC-BRL  SELL  FILLED  0.000077  337130.003917     0.000077   
34  1407068299  BTC-BRL  SELL  FILLED  0.000077  329167.995830     0.000077   
35  1407056031  BTC-BRL   BUY  FILLED  0.000077  325000.000000     0.000077   
36  1406519176  BTC-BRL   BUY  FILLED  0.000077  325737.268700     0.000077   

          cost           fee  limited  canceled origin           time_stamp  \
0     4.184352  1.000000e-08        1         0    API  2025-01-02 12:21:51   
1     5.236452  1.000000e-08        1         0    API  2025-01-02 12:21:35   
2     6.545881  2.000000e-08        1         0    API  2024-12-18 16:07:46   
3     8.181365  2.000000e-08        1         0    API  2024-12-18 15:42:06   
4    10.214051  3.000000e-08        1         0    API  2024-12-17 22:42:58   
5    12.752282  3.000000e-08        1         0    API  2024-12-17 12:17:48   
6    15.969600  4.000000e-08        1         0    API  2024-12-17 12:15:39   
7    19.957519  6.000000e-08        1         0    API  2024-12-17 12:14:04   
8   100.010008  2.000200e-01        1         0    API  2024-12-16 17:35:31   
9   100.447333  0.000000e+00        1         0    APP  2024-12-16 12:38:54   
10  100.645900  2.012910e-01        1         0    APP  2024-12-13 11:28:22   
11  100.581371  3.300000e-07        1         0    APP  2024-

id   market type  status    amount          price  exec_amount  \
0  1520875822  BTC-BRL  BUY  FILLED  0.000007  601200.000000     0.000007   
1  1520875579  BTC-BRL  BUY  FILLED  0.000009  601200.000000     0.000009   
2  1504227287  BTC-BRL  BUY  FILLED  0.000010  648107.029703     0.000010   
3  1504205222  BTC-BRL  BUY  FILLED  0.000013  650346.979332     0.000013   
4  1503401599  BTC-BRL  BUY  FILLED  0.000016  651822.016592     0.000016   
5  1502873638  BTC-BRL  BUY  FILLED  0.000019  660740.000000     0.000019   
6  1502871460  BTC-BRL  BUY  FILLED  0.000024  662639.004149     0.000024   
7  1502870034  BTC-BRL  BUY  FILLED  0.000030  663701.995344     0.000030   

        cost           fee  limited  canceled origin           time_stamp  \
0   4.184352  1.000000e-08        1         0    API  2025-01-02 12:21:51   
1   5.236452  1.000000e-08        1         0    API  2025-01-02 12:21:35   
2   6.545881  2.000000e-08        1         0    API  2024-12-18 16:07:46   
3   8.181365  2.000000e-08        1         0    API  2024-12-18 15:42:06   
4  10.214051  3.000000e-08        1         0    API  2024-12-17 22:42:58   
5  12.752282  3.000000e-08        1         0    API  2024-12-17 12:17:48   
6  15.969600  4.000000e-08        1         0    API  2024-12-17 12:15:39   
7  19.957519  6.000000e-08        1         0    API  2024-12-17 12:14:04   

             concluded  
0  2025-01-02 12:22:00  
1  2025-01-02 12:22:00  
2  2024-12-18 16:07:51  
3  2024-12-18 15:42:26  
4  2024-12-17 22:42:58  
5  2024-12-17 12:17:48  
6  2024-12-17 12:15:39  
7  2024-12-17 12:14:05

Preciso de um grafico que me mostre o melhor momento para comprar bitcoin

In [8]:
# Read the CSV file with error handling
try:
    df_bity = pd.read_csv(BTCBRL_BITY)

    # Display basic information about the dataframe
    print('DataFrame Info:')
    print(df_bity.info())
    print('\nFirst few rows:')
    display(df_bity.head())
    print('\nLast few rows:')
    display(df_bity.tail())

except Exception as e:
    print(f'Error reading file: {e}')
    # Try to identify the problematic rows
    with open(BTCBRL_BITY, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if len(line.split(',')) != len(file.readline().split(',')):
                print(f'Inconsistent columns at line {i + 1}')
                break

DataFrame Info:

: 

In [4]:
# Drop rows with any NaN values
df_bity = df_bity.dropna()

# Remove rows where any column has invalid values
df_bity = df_bity[df_bity['timestamp'].notna()]  # Ensure timestamp is not null
df_bity = df_bity[df_bity['close'] > 0]  # Remove rows with invalid prices

# Convert timestamp to datetime if it's not already
df_bity['timestamp'] = pd.to_datetime(df_bity['timestamp'])

# Sort by timestamp
df_bity = df_bity.sort_values('timestamp')

# Reset index
df_bity = df_bity.reset_index(drop=True)

# Save the cleaned DataFrame back to the same file
df_bity.to_csv(BTCBRL_BITY, index=False)

print(f'Cleaned DataFrame saved with {len(df_bity)} rows')

Cleaned DataFrame saved with 245334 rows

In [ ]:
orders_json = Orderbook().json()

pp(orders_json)

In [ ]:
# transformando em um dataframe pandas para ver no pygwalker
df = pd.DataFrame(orders_json)

# Expandir as colunas 'bids' e 'asks' em DataFrames separados
bids_df = pd.DataFrame(orders_json['bids'])
asks_df = pd.DataFrame(orders_json['asks'])

# Adicionar uma coluna para indicar o tipo de ordem
bids_df['type'] = 'bid'
asks_df['type'] = 'ask'

# Concatenar os DataFrames
orders_df = pd.concat([bids_df, asks_df], ignore_index=True)

# Exibir o DataFrame resultante
display(orders_df)

In [ ]:
pyg.walk(orders_df)

### Retorna uma lista de ordens já executadas.

Os retornos terão a seguinte estrutura:

type - Se a ordem é do tipo compra ou do tipo venda;
amount - Volume negociado na ordem;
market - Moedas envolvidas na ordem;
price - Preço do BTC no momento da ordem;
timestamp - Timestamp da ordem.

In [ ]:
trades_df = pd.DataFrame(Trades().json())

pyg.walk(trades_df)

In [ ]:
# Set the starting timestamp (e.g., September 1, 2017)
start_time = int(datetime(2017, 9, 1).timestamp() * 1000)

# Get the current timestamp
end_time = int(time.time() * 1000)

# Initialize a list to store data frames
data_frames = []

# Maximum interval per request (200 days in milliseconds)
max_interval = 200 * 24 * 60 * 60 * 1000

# Loop over the time range
current_time = start_time
while current_time < end_time:
    next_time = min(current_time + max_interval, end_time)
    df = get_klines(
        symbol='BTCBRL',
        interval='2h',
        startTime=current_time,
        endTime=next_time,
        limit=1500,
    )
    df['timestamp'] = pd.to_datetime(df['Kline open time'])
    df = df.rename(
        columns={
            'Close price': 'close',
            'High price': 'high',
            'Low price': 'low',
        }
    )
    data_frames.append(df)
    current_time = next_time
    time.sleep(1)  # Respect API rate limits

# Combine all data frames into one
full_df = pd.concat(data_frames, ignore_index=True)

# Save the data to a CSV file
full_df.to_csv(BTCBRL_FILE, index=False)

In [6]:
# Set the starting timestamp (e.g., September 1, 2017)
start_time = int(datetime(2017, 9, 1).timestamp())

# Get the current timestamp
end_time = int(time.time())

# Initialize a list to store data frames
data_frames = []

# Use the fixed interval of 1184400 seconds
interval = 1184400

# Loop over the time range
current_time = start_time
while current_time < end_time:
    try:
        next_time = min(current_time + interval, end_time)
        df = fetch_bitpreco_history(
            symbol='BTC_BRL',
            resolution='1',
            time_range={'from': current_time, 'to': next_time},
            countback=0,
            currency_code='BRL',
        )
        if df is not None:
            data_frames.append(df)
        current_time = next_time
        time.sleep(1)  # Respect API rate limits
    except Exception as e:
        print(f'Error at timestamp {current_time}: {e}')
        current_time += interval
        continue

# Combine all data frames into one
if data_frames:
    full_df = pd.concat(data_frames, ignore_index=True)
    # Save the data to a CSV file
    full_df.to_csv(BTCBRL_BITY, index=False)
else:
    print('No data was collected')

Erro HTTP: 400

Conteúdo da página:

{"s":"error","errmsg":"sizeCalculation return invalid (expect positive integer)"}

Erro HTTP: 400

Conteúdo da página:

...

Conteúdo da página:

{"s":"error","errmsg":"sizeCalculation return invalid (expect positive integer)"}

Erro HTTP: 400

Conteúdo da página:

{"s":"error","errmsg":"sizeCalculation return invalid (expect positive integer)"}

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(
#     data=[
#         go.Candlestick(
#             x=data['timestamp'],
#             open=data['open'],
#             high=data['high'],
#             low=data['low'],
#             close=data['close'],
#         )
#     ]
# )
# fig.show()